# enriching Mind

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# New call
from pipeline_newcall import generate_groups
import sys
import pickle, json
import numpy as np
from scipy.spatial.distance import cosine
from collections import Counter
sys.path.append("/home/ray__/ssd/ai-engine/pkg/")
import text_preprocessing.preprocess as tp
from extra_preprocess import preprocess_text

sys.path.append("helper_functions/")
from generate_controlled_top_10_ent import get_ent

sys.path.append("/home/ray__/ssd/BERT/")
from gpt_feat_utils import GPT_Inference
gpt_model = GPT_Inference("/home/ray__/ssd/BERT/models/se/epoch3/", device="cpu")

# S.E
ent_fv_full = pickle.load(open("/home/ray__/ssd/minds/se/se_entity_feats_v3.pkl","rb"))
sent_dict = pickle.load(open("/home/ray__/ssd/minds/se/se_sent_dict_v3.pkl", "rb"))
com_map = pickle.load(open("/home/ray__/ssd/minds/se/com_map.pkl", "rb"))
kp_entity_graph = pickle.load(open("/home/ray__/ssd/minds/se/deployment/kp_entity_graph.pkl", "rb"))


common_entities = ent_fv_full.keys() & com_map.keys()
ent_fv = {}
for ent in common_entities:
    ent_fv[ent] = ent_fv_full[ent]
    


ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.
[nltk_data] Downloading package wordnet to /tmp/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /tmp/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /tmp/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.


{"level": "warning", "filename": "tokenization_openai.py", "lineno": 96, "module": "tokenization_openai", "ts": "2020-02-14T10:58:44.465357Z", "msg": "ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy."}
{"level": "info", "filename": "tokenization_utils.py", "lineno": 416, "module": "tokenization_utils", "ts": "2020-02-14T10:58:44.649077Z", "msg": "Adding _start_ to the vocabulary"}
{"level": "info", "filename": "tokenization_utils.py", "lineno": 416, "module": "tokenization_utils", "ts": "2020-02-14T10:58:44.650003Z", "msg": "Adding _delimiter_ to the vocabulary"}
{"level": "info", "filename": "tokenization_utils.py", "lineno": 416, "module": "tokenization_utils", "ts": "2020-02-14T10:58:44.651254Z", "msg": "Adding _classify_ to the vocabulary"}
{"level": "warning", "filename": "tokenization_openai.py", "lineno": 96, "module": "tokenization_openai", "ts": "2020-02-14T10:58:47.563327Z", "msg": "ftfy or spacy is not installed using BERT BasicTokenizer inst

In [39]:
from copy import deepcopy
def get_ranked_groups(request, ent_fv, com_map, kp_entity_graph):
    group, group_ent = get_ent(request, ent_fv, com_map, kp_entity_graph)
    return group_ent

def get_entity_mapping(group_ent, ent_fv, com_map, kp_entity_graph):
    group_ent_map = {}
    for groupid, ent_list in group_ent.items():
        group_ent_map[groupid] = [com_map[ent] for ent in list(map(lambda kv:kv[0], group_ent[groupid]))]
        #group_ent_score[groupid] = [ranked_com[com] for com in group_ent_score[groupid] if com in ranked_com.keys()]
    
    return group_ent_map

def filter_entity_community(group_ent_map, ent_fv, com_map, kp_entity_graph):
    group_ent_map_filtered_intrm = {}
    group_ent_map_filtered = {}
    for groupid, ent_map in group_ent_map.items():
        filtered_ent_map = []
        if len(set(ent_map)) == len(ent_map):
            group_ent_map_filtered[groupid] = []
        else:
            count_a = Counter(ent_map).most_common()
            for i, count in count_a:
                if count>1 :
                    filtered_ent_map.append((i, count))

            group_ent_map_filtered[groupid] = filtered_ent_map

    return group_ent_map_filtered


def get_entity_mapping_rank(group_ent_map_filtered, gc, lc, ent_fv, com_map, kp_entity_graph):
    group_ent_map_rank_lc = {}
    group_ent_map_rank_gc = {}
    for groupid, ent_map_list in group_ent_map_filtered.items():
        group_ent_map_rank_intrm_lc = []
        group_ent_map_rank_intrm_gc = []
        
        for ent_map, count in ent_map_list:
            if ent_map in lc.keys() and sum(lc[ent_map])!=0:
                group_ent_map_rank_intrm_lc.append(sum(lc[ent_map]))
            else:
                if ent_map in gc.keys():
                    group_ent_map_rank_intrm_gc.append(gc[ent_map])
                else:
                    group_ent_map_rank_intrm_gc.append(0)
        if group_ent_map_rank_intrm_lc!=[]:
            group_ent_map_rank_lc[groupid] = sum(group_ent_map_rank_intrm_lc)
        else:
            group_ent_map_rank_gc[groupid] = sum(group_ent_map_rank_intrm_gc) 
            
            
    ## update gc and lc
    updated_lc_list = []
    for groupid, ent_map_list in group_ent_map_filtered.items():
        for ent_map, count in ent_map_list:
            if ent_map in lc.keys():
                if len(lc[ent_map])!=5:
                    lc[ent_map].append(count)
                else:
                    del lc[ent_map][0]
                    lc[ent_map].append(count)
                updated_lc_list.append(ent_map)
            else:
                lc[ent_map] = [count]
                updated_lc_list.append(ent_map)
                
            if ent_map in gc.keys():
                gc[ent_map] +=count
            else:
                gc[ent_map] = count
                
    print (updated_lc_list)
    lc_copy = deepcopy(list(lc.items()))            
    for ent, freq in lc_copy:
        if ent not in updated_lc_list:
            if sum(lc[ent]) == 0:
                del lc[ent]
            else:
                if len(lc[ent])!=5:
                    lc[ent].append(0)
                else:
                    del lc[ent][0]
                    lc[ent].append(0)
    
    return group_ent_map_rank_lc, group_ent_map_rank_gc, gc, lc 


def compute_groups_new_call(req, ent_fv, com_map, kp_entity_graph, gc, lc, store=False):
    group, group_ent = get_ranked_groups(req, ent_fv, com_map, kp_entity_graph)
    group_ent_map = get_entity_mapping(group_ent, ent_fv, com_map, kp_entity_graph)
    group_ent_map_filtered = filter_entity_community(group_ent_map, ent_fv, com_map, kp_entity_graph)
    group_ent_map_rank_lc, group_ent_map_rank_gc, gc_copy, lc_copy = get_entity_mapping_rank(group_ent_map_filtered, gc, lc, ent_fv, com_map, kp_entity_graph)
    if store:
        print ("writing the gc and lc update.")
        pickle.dumps(gc, open("/home/ray__/ssd/minds/se/recency/gc.pkl","wb"))
        pickle.dumps(lc, open("/home/ray__/ssd/minds/se/recency/lc.pkl","wb"))
    return group, group_ent_map_rank_lc, group_ent_map_rank_gc, gc_copy ,lc_copy

In [40]:
test_set = ['topic_testing/sync_eng_2020_01_10.txt']
for file in test_set:
    gc = pickle.load(open("/home/ray__/ssd/minds/se/recency/gc.pkl","rb"))
    lc = pickle.load(open("/home/ray__/ssd/minds/se/recency/lc.pkl","rb"))
    with open(file,'rb') as f:
        request = json.load(f)
        if isinstance(request, str):
            request = json.loads(request)

    #ranked_groups, group_map, group_rank, group_ent, gc, lc = compute_groups_new_call(request)
    group, group_ent_map_rank_lc, group_ent_map_rank_gc, gc, lc = compute_groups_new_call(request, ent_fv, com_map, kp_entity_graph, gc, lc)

Using  01DAAYHEKY5F4E02QVRJPTFTXV  for feature extraction
{"level": "info", "filename": "scorer.py", "lineno": 122, "module": "scorer", "batches count": 1, "number of sentences": 149, "ts": "2020-02-14T12:35:06.110011Z", "msg": "computing in batches"}
{"level": "info", "filename": "scorer.py", "lineno": 125, "module": "scorer", "iteration count:": 0, "ts": "2020-02-14T12:35:06.112174Z", "msg": "getting feature vector"}
{"level": "info", "filename": "scorer.py", "lineno": 133, "module": "scorer", "iteration count": 0, "ts": "2020-02-14T12:35:13.768018Z", "msg": "Request Sent"}
('Hundred hundred add to my question was when you remember what he had to do handle the case of hanging a call when you are past or something like that.', '2020-01-10T06:01:04Z', '8d6db5f7d9b74c54ba38fe710ffcaf3f', '003af464-62b3-4566-afe0-97347acfa9bb') ('Diesel column so that the solvent should that so my question is did we hand we have to handle that case properly testing now.', '2020-01-10T06:01:24Z', '8d6db5f

{"level": "info", "filename": "grouper_segments.py", "lineno": 397, "module": "grouper_segments", "outlier threshold is : ": 0.5341182351112366, "ts": "2020-02-14T12:35:15.911729Z", "msg": "Outlier Score"}
{"level": "info", "filename": "grouper_segments.py", "lineno": 1041, "module": "grouper_segments", "edges before prunning": 603, "edges after prunning": 603, "modularity": 0.7195835443202092, "ts": "2020-02-14T12:35:16.929873Z", "msg": "Meeting Graph results"}
cluster before alteration=========>
Hundred hundred add to my question was when you remember what he had to do handle the case of hanging a call when you are past or something like that.
So lies and deception has raised the pr for storing all the he said you have to share he has just having multiple scenarios editing the event and consoling my mother is things seem to be working fine.
Diesel column so that the solvent should that so my question is did we hand we have to handle that case properly testing now.
Looking into the li

Relevant sentence:  Okay, so we should be able to find a fix for this aggressive behavior.    =====    I mean the twins will not blindly go by it self interest in even though we start and then observe the behavior will have to find an alternative for this approach both.
order difference: 0
Relevant sentence:  Uhhuh for the first call we go with the main domain with as I said every mind would be associated with the key phrase graph the default Keepsake.    =====    That is the domain key phrase graph that means for now.
Not Relevant sentence:  That is the domain key phrase graph that means for now.    !=    So initially, we have only default mine and then one single ether mind.
order difference: 11
order difference: 0
Relevant sentence:  So initially, we have only default mine and then one single ether mind.    =====    So a bit from my side the main thing is like the enrichment of the mines so initially, right right now we have some kind of a ranking in place for all those default Mind




====================Group Cluster=========================
[[["But it is required forecast. Okay. It is a prerequisite for completing the entire communica. Okay. All right, finish the progress bar minute. That's the next thing a little softer. The most of the modal dialog is small we can do because we have like a bunch of chemicals, right? So even if it I understand the conflict the the commission issue, but what is like given so you want me to assist the number but something simple some like can we what are what will happen? For example, if he appointed date like today's college like 10 Jan 29 2012 10 years something so I know II just did 20. I know that I need to also put the same name and the same day. There will be a pollution. I can put the time also, that's okay then it's like a number less. My point is why don't you just take a chance and just from the date? Hope you don't know that you - repeating same data. You can collect multiple meetings same at the same time is when we 

In [42]:
gc

{4: 16,
 89: 5,
 8: 21,
 35: 22,
 195: 3,
 318: 4,
 5: 1,
 168: 1,
 146: 4,
 14: 45,
 204: 2,
 599: 1,
 1508: 1,
 776: 2,
 264: 3,
 233: 2,
 106: 1,
 37: 1,
 215: 1,
 98: 2,
 7: 32,
 33: 44,
 1553: 1,
 201: 2,
 66: 1,
 1090: 1,
 866: 2,
 28: 125,
 15: 1,
 108: 1,
 281: 1,
 1: 49,
 49: 60,
 537: 1,
 21: 25,
 50: 5,
 470: 1,
 17: 14,
 57: 27,
 121: 1,
 93: 2,
 56: 1,
 363: 1,
 30: 2,
 512: 1,
 199: 3,
 838: 2,
 76: 3,
 358: 1,
 186: 1,
 41: 1,
 11: 83,
 896: 2,
 699: 2,
 357: 2,
 58: 5,
 48: 2,
 0: 2,
 148: 2}

In [18]:
lc

{4: [4, 2],
 89: [],
 8: [],
 35: [2],
 195: [],
 318: [],
 5: [],
 168: [],
 146: [],
 14: [3, 2, 3, 2, 2],
 204: [],
 599: [],
 1508: [],
 776: [],
 264: [],
 233: [],
 106: [],
 37: [],
 215: [],
 98: [],
 7: [2, 2, 2, 4, 2],
 33: [3, 3, 3, 2, 10],
 1553: [],
 201: [],
 66: [],
 1090: [],
 866: [],
 28: [3, 4, 7, 4, 2],
 15: [],
 108: [],
 281: [],
 1: [2],
 49: [3, 13, 4, 4, 10],
 537: [],
 21: [2, 7, 2, 11, 2],
 50: [],
 470: [],
 17: [3, 2],
 57: [2, 3, 4, 7, 3],
 121: [],
 93: [],
 56: [],
 363: [],
 30: [3],
 512: [],
 199: [],
 838: [],
 76: [],
 358: [],
 186: [],
 41: [],
 11: [7, 37, 3, 7, 3],
 896: [2],
 357: [2, 2],
 443: [2],
 64: [2],
 58: [5],
 48: [2],
 61: [3]}

In [41]:
lc

{4: [4],
 35: [4],
 14: [2, 3, 3],
 7: [2, 6, 2, 4, 2],
 33: [9, 4, 3, 10],
 28: [8, 5, 7, 4, 2],
 1: [5, 2],
 49: [3, 13, 5, 4, 10],
 21: [7, 11, 2, 2],
 17: [2],
 57: [2, 2, 4, 7, 3],
 11: [6, 37, 3, 7, 3],
 896: [2],
 699: [2],
 357: [2],
 58: [5],
 48: [2],
 0: [2],
 148: [2]}